In [4]:
from flowgen import stock_5y, spectral_entropy
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
import numpy as np

In [14]:
aapl = stock_5y('^GSPC')

In [ ]:
ts_data = aapl.set_index('date')['close']

In [3]:
px.line(aapl, x='date', y='close')

# Long Range Dependence

In [ ]:
# segment length to calculate local fluctuations
import math
q = math.pow((314/4), (1/9))

T = [int(4 * math.pow(q, j)) for j in range(10)]
T[9] = 314

T = np.array(T)

### Detrended Fluctuations Analysis of Real Time Series

##### Interpretation:
##### alpha = 1 : perfect similarity
##### alpha = 1/2 : uncorrelated, white noise
##### 1/2 < alpha < 1 : positive correlation
##### alpha < 1/2 : inversely correlated
##### alpha = 1 : non-stationarity, while correlation exist they cannot be described in the form of power law relationship
##### alpha = 3/2 : Brown noise

In [ ]:
import fathon

dfa = fathon.DFA(aapl['close'])
n, local_fluctuation = np.array(dfa.computeFlucVec(T))
alpha_real, intercept = dfa.fitFlucVec()
alpha_real

### Synthetic Time Series

In [6]:
aapl_synth = pd.read_csv('Synthetic_data_1/AAPL_synth1.csv')
synth_v_real = pd.DataFrame({'Synthetic': aapl_synth['close\n'], 'Actual': aapl['close']})

In [ ]:
aapl_synth.set_index('date')

In [7]:
# Plot of Actual vs Synthetic
fig = px.line(synth_v_real, y=synth_v_real.columns,
             title='Actual vs Synthetic')
fig.show()

### DFA on Sythetic Time Series

In [ ]:
dfa = fathon.DFA(aapl_synth['close\n'])
n, local_fluctuation = np.array(dfa.computeFlucVec(T))
alpha_synth, intercept = dfa.fitFlucVec()
alpha_synth

# Complexity

### Spectral Entropy (measure the uniformity of the power spectrum)

In [8]:
spectral_entropy(aapl_synth['close\n'])

0.042282420142377534

In [9]:
returns = np.log(aapl_synth['close\n'] / aapl_synth['close\n'].shift(1))
returns.dropna(inplace=True)
spectral_entropy(returns)

0.924028692290308

# Normality
### Kurtosis

In [10]:
def kurtosis(signal: np.ndarray) -> float:
    def central_moment(order: int):
        return np.mean((signal - signal.mean()) ** order)

    return central_moment(4) / central_moment(2) ** 2

In [16]:
kurtosis(aapl_synth['close\n'])

2.445538216966718

#### Skewness

In [17]:
def skewness(signal: np.ndarray) -> float:
    def central_moment(order: int):
        return np.mean((signal - signal.mean()) ** order)

    return central_moment(3) / central_moment(2) ** (3 / 2)

In [18]:
skewness(aapl_synth['close\n'])

0.5423652670289326

### Gaussianity of the Differences(GoD)

In [21]:
from scipy import stats

def god(signal: np.ndarray) -> float:
    """
    Measure how Gaussian (normal) the first-order differences are.
    Returns a value between 0 and 1, where 1 = perfectly Gaussian.
    """
    # Step 1: Calculate first-order differences
    diffs = np.diff(signal)

    # Step 2: Standardize the differences (mean=0, std=1)
    diffs_standardized = (diffs - np.mean(diffs)) / np.std(diffs)

    # Step 3: Use Shapiro-Wilk test or Anderson-Darling test
    # Option A: Shapiro-Wilk (works well for smaller samples)
    statistic, p_value = stats.shapiro(diffs_standardized)

    # Return p-value as gaussianity measure (high p-value = more Gaussian)
    return p_value

In [22]:
god(aapl_synth['close\n'])

3.162199207093656e-44